# Part 5: Predicting for May's 1st bidding exercise

Do note that you have to use an environment with Tensorflow installed to run this notebook.

In this section, I saw an opportunity to test my model on a totally unseen data which is the 1st bidding exercise for May 2021. Hence, I will perform a prediction to s

In [1]:
# Importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

from sklearn.metrics import mean_squared_error

from sklearn.model_selection import (
    train_test_split,
)

from sklearn.preprocessing import (
    StandardScaler,
    MinMaxScaler
)

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GRU, LSTM
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

In [2]:
# Importing dataset
model_cat_a = pd.read_csv('../data/merged_cat_a.csv')

In [3]:
# Convert dates to datetime format
model_cat_a['new_date'] = pd.to_datetime(model_cat_a['new_date'])

In [4]:
# Creating separate df with date as index
model_cat_a_rnn = model_cat_a.set_index('new_date')
model_cat_a_rnn.sort_index(inplace=True)

In [6]:
model_cat_a_rnn

,bidding_exercise,date,year,category,premium,quota,total_bids_received,number_of_successful_bids,quarter,month,...,total_pop,pop_density,total_pop_growth,average_income,median_income,average_employed_income,median_employed_income,close,air_departures,birth_rate
new_date,,,,,,,,,,,,,,,,,,,,,
2002-04-01,April 2002 First Open Bidding Exercise,2002-04-01 00:00:00,2002,cat_a,37201.0,1110.0,2484.0,1092.0,2,4,...,4175950.0,6078.0,0.9,5069.0,3628.0,5572.0,4038.0,1809.08,1101439.0,11.4
2002-04-15,April 2002 Second Open Bidding Exercise,2002-04-15 00:00:00,2002,cat_a,36000.0,1149.0,1890.0,1148.0,2,4,...,4175950.0,6078.0,0.9,5069.0,3628.0,5572.0,4038.0,1757.31,1101439.0,11.4
2002-05-01,May 2002 First Open Bidding Exercise,2002-05-01 00:00:00,2002,cat_a,35000.0,1128.0,1705.0,1127.0,2,5,...,4175950.0,6078.0,0.9,5069.0,3628.0,5572.0,4038.0,NaN,1155074.0,11.4
2002-05-15,May 2002 Second Open Bidding Exercise,2002-05-15 00:00:00,2002,cat_a,33401.0,1111.0,1320.0,1080.0,2,5,...,4175950.0,6078.0,0.9,5069.0,3628.0,5572.0,4038.0,1745.87,1155074.0,11.4
2002-06-01,June 2002 First Open Bidding Exercise,2002-06-01 00:00:00,2002,cat_a,33009.0,1111.0,1382.0,1105.0,2,6,...,4175950.0,6078.0,0.9,5069.0,3628.0,5572.0,4038.0,NaN,1127989.0,11.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-02-15,February 2022 Second Open Bidding Exercise,2022-02-23 00:00:00,2022,cat_a,63000.0,536.0,876.0,524.0,1,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3421.38,334406.0,NaN
2022-03-01,March 2022 First Open Bidding Exercise,2022-03-09 00:00:00,2022,cat_a,68501.0,531.0,962.0,526.0,1,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3278.63,565823.0,NaN
2022-03-15,March 2022 Second Open Bidding Exercise,2022-03-23 00:00:00,2022,cat_a,71556.0,543.0,849.0,542.0,1,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3236.04,565823.0,NaN


In [8]:
# Scaling the data
scaler = MinMaxScaler()
df1 = scaler.fit_transform(np.array(model_cat_a_rnn['premium']).reshape(-1, 1))

In [9]:
training_size = int(len(df1))

In [10]:
training_size

482

In [11]:
# Training the data on the whole dataset
train_data = df1[0:training_size, :]

In [12]:
train_data

array([[0.40390671],
       [0.39086625],
       [0.38000825],
       [0.36264631],
       [0.35838998],
       [0.32876935],
       [0.34099546],
       [0.33437208],
       [0.32885622],
       [0.33971422],
       [0.33115811],
       [0.30844318],
       [0.34020283],
       [0.33394862],
       [0.31608721],
       [0.31494712],
       [0.30944212],
       [0.29306825],
       [0.30931182],
       [0.2877044 ],
       [0.33660883],
       [0.3257074 ],
       [0.30734652],
       [0.2931334 ],
       [0.29854068],
       [0.30458859],
       [0.3181068 ],
       [0.33760777],
       [0.3452518 ],
       [0.34135378],
       [0.34829204],
       [0.34201611],
       [0.33111468],
       [0.33645682],
       [0.2715477 ],
       [0.33254794],
       [0.31393733],
       [0.30725966],
       [0.28595626],
       [0.29015831],
       [0.26492432],
       [0.26329562],
       [0.25840952],
       [0.26058112],
       [0.26587982],
       [0.26479402],
       [0.26705249],
       [0.270

In [13]:
# Creating a function to have create the X and y for modelling
def create_dataset(df, time_step=1):
    X, y = [], []

    for i in range(len(df) - time_step):
        new_values = df[i:(i + time_step), 0]
        X.append(new_values)
        
        y.append(df[i + time_step, 0])
    return np.array(X), np.array(y)

In [14]:
# Creating X and y for both training and test data with 18 timsteps
X_train, y_train = create_dataset(train_data, 18)

In [15]:
# reshape input to be [samples, time steps, features]
X_train = np.reshape(X_train,(X_train.shape[0], X_train.shape[1], 1)) #adding the batch_size axis
X_train.shape

(464, 18, 1)

In [16]:
X_train.shape, y_train.shape

((464, 18, 1), (464,))

In [28]:
# Instantiating the model
model = Sequential()
model.add(GRU(
    16,
    input_shape=(18, 1),
    return_sequences=True,
))

model.add(GRU(
    16,
    return_sequences=True,
))

model.add(GRU(
    16,
    return_sequences=True,
))

model.add(GRU(
    16,
    return_sequences=False,
))

model.add(Dense(4, activation='relu'))
model.add(Dense(1,))

In [29]:
# Compiling the model
model.compile(
    optimizer='adam',
    loss='mse',
    metrics=['mae']
)

In [30]:
# Fitting the model
history = model.fit(
    X_train,
    y_train,
    batch_size=16,
    epochs=50
)

Train on 464 samples
Epoch 1/50


2022-05-11 10:48:31.311974: W tensorflow/core/grappler/optimizers/implementation_selector.cc:310] Skipping optimization due to error while loading function libraries: Invalid argument: Functions '__inference___backward_standard_gru_23338_23712_specialized_for_StatefulPartitionedCall_1_at___inference_distributed_function_24665' and '__inference___backward_cudnn_gru_with_fallback_22283_22421' both implement 'gru_bdb78f35-8704-4a4e-8f7e-bb5d51326f0f' but their signatures do not match.


464/464 [==============================] - 11s 23ms/sample - loss: 0.0399 - mae: 0.1458
Epoch 2/50
464/464 [==============================] - 2s 5ms/sample - loss: 0.0068 - mae: 0.0634
Epoch 3/50
464/464 [==============================] - 2s 5ms/sample - loss: 0.0032 - mae: 0.0427
Epoch 4/50
464/464 [==============================] - 2s 5ms/sample - loss: 0.0031 - mae: 0.0425
Epoch 5/50
464/464 [==============================] - 2s 5ms/sample - loss: 0.0028 - mae: 0.0389
Epoch 6/50
464/464 [==============================] - 2s 5ms/sample - loss: 0.0028 - mae: 0.0396
Epoch 7/50
464/464 [==============================] - 2s 5ms/sample - loss: 0.0025 - mae: 0.0373
Epoch 8/50
464/464 [==============================] - 2s 5ms/sample - loss: 0.0030 - mae: 0.0410
Epoch 9/50
464/464 [==============================] - 2s 5ms/sample - loss: 0.0023 - mae: 0.0350
Epoch 10/50
464/464 [==============================] - 2s 5ms/sample - loss: 0.0022 - mae: 0.0350
Epoch 11/50
464/464 [=================

In [32]:
# Getting the last 18 values to use as the test set
n = 18
test_data = df1[int(len(df1) - n):len(df1),:1]

In [33]:
test_data.shape

(18, 1)

In [34]:
# reshape input to be [samples, time steps, features]
test_input = test_data.reshape((1, 18, 1))
test_input.shape

(1, 18, 1)

In [35]:
# Checking on the test input
test_input

array([[[0.4906404 ],
        [0.5069274 ],
        [0.51030424],
        [0.52116224],
        [0.5103151 ],
        [0.57229256],
        [0.58315056],
        [0.59717909],
        [0.61899281],
        [0.63843949],
        [0.62538817],
        [0.62863472],
        [0.65972117],
        [0.68403223],
        [0.74376208],
        [0.77693327],
        [0.79256879],
        [0.74591196]]])

In [36]:
# Predicting for the unseen data
prediction = scaler.inverse_transform(model.predict(test_input))
prediction

2022-05-11 10:51:49.733337: W tensorflow/core/grappler/optimizers/implementation_selector.cc:310] Skipping optimization due to error while loading function libraries: Invalid argument: Functions '__inference_standard_gru_30167' and '__inference_standard_gru_30167_specialized_for_sequential_1_gru_6_StatefulPartitionedCall_at___inference_distributed_function_30818' both implement 'gru_1e96a9c7-89da-4434-a6f8-2b435e1a1aed' but their signatures do not match.


array([[69825.625]], dtype=float32)

In [38]:
# Adding the increment to ensure 90% success rate
prediction + 1943

array([[71768.625]], dtype=float32)

The predicted premium is $71,768. 

The actual premium for the 1st May Bidding exercise for cat A quotas is $70,901.

Our Model was able to accurately predict the premium and successfully attained the COE!.

## Proceed to the next notebook on the prediction for other categories.